## Предобработка

Функции предобработки для одного документа и для всего корпуса

In [1]:
import os
from tqdm.notebook import tqdm
from data_load import raw_data_load # Своя функция для загрузки исходных датасетов

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import SnowballStemmer
from razdel import tokenize

In [2]:
# nltk.download('stopwords')  # Будут предупреждения, если пакет уже загружен - это нормально
russian_stopwords = set(stopwords.words('russian'))
stemmer = SnowballStemmer("russian")

# Эмоционально значимые комбинации символов
emoticon_patterns = [
    r'\:\)', 
    r'\:\(', 
    r'\;\)', 
    r'\.\.\.',
    r'\(', 
    r'\)'
]

def preprocess_text(text: str) -> str:
    """
    Предобработка текста со стеммингом, учетом одиноких скобок, эмоциональных комбинаций и многоточий.
    :param text: Строка для обработки
    :return: Строка из обработанных токенов
    """
    
    text = text.lower()

    for pattern in emoticon_patterns:
        text = re.sub(re.escape(pattern), f' {pattern} ', text)
    
    # Удаление ссылок, упоминаний и лишних символов
    text = re.sub(r'http\S+|www\S+|@\S+|#\S+', '', text)
    text = re.sub(r'\d+|№', '', text)
    
    # Сохранение эмоционально значимой пунктуации и одиночных скобок
    text = text.replace('!', ' ! ').replace('?', ' ? ')
    
    # Оставляем одинокие скобки, многоточие и эмоциональные комбинации
    text = re.sub(r'[^()\w\s!?\.]', '', text)  

    tokens = [token.text for token in tokenize(text) if token.text.isalpha() or token in {')', '(', '!', '?', '...', '!?'}]

    processed_tokens = [
        stemmer.stem(token) 
        for token in tokens
        if token not in russian_stopwords or token in {'не', 'нет'}
    ]
    
    return ' '.join(processed_tokens)


def preprocess(dataset: pd.DataFrame) -> pd.DataFrame:
    """
    Создание дополнительного признака в датасете - результат применения preprocess_text к тексту.
    :param dataset: Датасет, который надо обработать
    :return: Обработанный датасет с новым столбцом - обработанным текстом
    """

    tqdm.pandas(desc="Обработка датасета")

    dataset["prep_text"] = dataset["text"].progress_apply(preprocess_text)

    return None

Применение функции предобработки к датасетам и их сохранение в зависимости от того, было ли это сделано ранее.

In [3]:
train, test = raw_data_load()
train.drop(columns=["Unnamed: 0"], inplace=True)
test.drop(columns=["Unnamed: 0"], inplace=True)

data_dir = "../data"

prep_train_path = os.path.join(data_dir, 'prep_train.csv')
prep_test_path = os.path.join(data_dir, 'prep_test.csv')

if not os.path.exists(prep_train_path):
    print("Обработка тренировочного датасета")
    preprocess(train)
    test.dropna(subset=["prep_text"], inplace=True)
    train.to_csv(prep_train_path, index=False)  # Сохранение обработанных данных
    print(f"Обработанные тренировочные данные сохранены")

if not os.path.exists(prep_test_path):
    print("Обработка тестового датасета")
    preprocess(test)
    test.dropna(subset=["prep_text"], inplace=True)
    test.to_csv(prep_test_path, index=False)  # Сохранение обработанных данных
    print(f"Обработанные тестовые данные сохранены")

Обработка тренировочного датасета


Обработка датасета:   0%|          | 0/189891 [00:00<?, ?it/s]

Обработанные тренировочные данные сохранены
Обработка тестового датасета


Обработка датасета:   0%|          | 0/21098 [00:00<?, ?it/s]

Обработанные тестовые данные сохранены
